# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235029049326                   -0.50    9.0         
  2   -7.250209692573       -1.82       -1.38    1.0   8.24ms
  3   -7.250943453901       -3.13       -1.83    1.0   8.33ms
  4   -7.251200646194       -3.59       -2.03    2.0   9.76ms
  5   -7.251315430371       -3.94       -2.47    1.0   8.26ms
  6   -7.251337513610       -4.66       -3.12    1.0   8.11ms
  7   -7.251338614903       -5.96       -3.58    2.0   9.74ms
  8   -7.251338773108       -6.80       -3.83    2.0   10.1ms
  9   -7.251338795565       -7.65       -4.41    1.0   8.38ms
 10   -7.251338798439       -8.54       -4.83    3.0   11.2ms
 11   -7.251338798637       -9.70       -5.11    2.0   10.2ms
 12   -7.251338798692      -10.26       -5.51    1.0   8.63ms
 13   -7.251338798703      -10.94       -5.97    2.0   10.1ms
 14   -7.251338798704      -11.89       -6.38    1.0   8.61ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.039032014088467674
[ Info: Arnoldi iteration step 2: normres = 0.5689632877500937
[ Info: Arnoldi iteration step 3: normres = 0.7953435848175489
[ Info: Arnoldi iteration step 4: normres = 0.43670950611388387
[ Info: Arnoldi iteration step 5: normres = 0.3425572003185042
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.57e-02, 5.59e-02, 2.43e-01, 2.26e-01, 3.73e-02)
[ Info: Arnoldi iteration step 6: normres = 0.44544168137252355
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.94e-02, 7.36e-02, 4.15e-01, 1.14e-01, 7.40e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08274573234156002
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (8.80e-04, 1.15e-02, 3.19e-02, 4.20e-02, 5.51e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10867115177238572
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.17